In [1]:
import sqlite3
import pandas as pd

In [2]:
# read the updated thing
new_df = pd.read_csv('table_to_make_cards.csv', sep=',')
new_df = new_df[['cast_note', 'cat_note', 'cat_tags']]
n1 = len(new_df.index)

In [3]:
# Step 1: Connect to the SQLite database
conn = sqlite3.connect('collection.anki21')
cursor = conn.cursor()

In [4]:
# Step 2: Read the table into a pandas DataFrame
table_name = 'notes'  # Replace with your table name
df = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)

df = df.merge(new_df, how='inner', 
              left_on='sfld', 
              right_on='cast_note')
n2 = len(df.index)
assert n1 == n2

In [5]:
# rename a couple of columns and remove the others
df.drop(['cast_note', 'sfld', 'tags'], axis=1, inplace=True)
df.rename({'cat_note': 'sfld',
           'cat_tags': 'tags'},
          axis=1, inplace=True)

In [6]:
# make all flds ""
df['uuid'] = df['flds'].str.split(' ', expand=True)[0]
df['flds'] = ""

In [10]:
df.to_csv('table_to_make_cards_2.csv', sep='|')
df.head(1).to_csv('table_to_make_cards_2_test.csv', sep='|')

In [8]:
df.columns

Index(['id', 'guid', 'mid', 'mod', 'usn', 'flds', 'csum', 'flags', 'data',
       'sfld', 'tags', 'uuid'],
      dtype='object')

In [9]:
# # Step 3: Modify the DataFrame
# # Example: Add a new column or make any modifications
# df['new_column'] = df['existing_column'] * 2  # Replace with your logic

# print("Modified table:")
# print(df)

# Step 4: Replace the table in the database
# First, drop the existing table (if it exists)
cur = conn.cursor()
cur.execute(f"DROP TABLE IF EXISTS {table_name}")

# Then, write the modified DataFrame back to the database
df.to_sql(table_name, conn, index=False, if_exists='replace')  # Replace mode

# Step 5: Verify the changes (optional)
df_updated = pd.read_sql_query(f"SELECT * FROM {table_name}", conn)
print("Updated table in the database:")
print(df_updated)

# Step 6: Close the database connection
conn.close()

Updated table in the database:
                 id        guid            mid         mod  usn flds  \
0     1599309962954  t1||z{8OP4  1596992551063  1599310148    0        
1     1599309962955  PhbeK,#2h}  1596992551063  1599309962    0        
2     1599309962956  m05T}gc0C;  1596992551063  1599309962    0        
3     1599309962957   |wzu@j5%J  1596992551063  1599309962    0        
4     1599309962959  F(KURcB8Xf  1596992551063  1599309962    0        
...             ...         ...            ...         ...  ...  ...   
2781  1599309967179  k8%N~;6SBK  1596992551063  1599309963    0        
2782  1599309967182  NV-TtTh<UU  1596992551063  1599309963    0        
2783  1599309967183  QSpm$`>:+r  1596992551063  1599309963    0        
2784  1599309967184  r8cXLGlsvj  1596992551063  1599309963    0        
2785  1599309967185  lP{Uu4aLm1  1596992551063  1599309963    0        

            csum  flags data  \
0     4184801480      0        
1      851395593      0        
2      3